In [1]:
import os
os.environ['http_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128' 
os.environ['https_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128'
os.chdir('../')
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/Text_Summarization_HF'

STEPS TO FOLLOW: 
- config.yaml
- ~~params.yaml~~
- entity
- configuration manager in src config
- components
- pipeline
- main.py
- app.py


In [2]:
# Start with the ENTITY file which will include params as well
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_filename: Path

In [3]:
# Write the ConfigurationManager
from txtsummarizer.constants import *
from txtsummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILEPATH,
        params_filepath = PARAMS_FILEPATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create artifacts folder
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        # Create model evaluation directory
        create_directories([config.root_dir])

        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_filename=config.metric_filename
        )

        return model_eval_config

In [4]:
# Write the components file
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric

class ModelEvaluation:
    def __init__(
        self,
        config: ModelEvaluationConfig
    ):
        self.config = config

    def generate_data_chunks(self, list_of_elements, batch_size):
        """
        Splits the dataset into smaller batches
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric(
        self,
        dataset,
        metric, 
        model, 
        tokenizer,     
        batch_size=16, 
        device='cuda' if torch.cuda.is_available() else 'cpu', 
        column_text="article", 
        column_summary="highlights"
        ):
        article_batches = list(self.generate_data_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_data_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(
                article_batch, 
                max_length=1024,  
                truncation=True, 
                padding="max_length", 
                return_tensors="pt"
                )
            
            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device), 
                length_penalty=0.8, 
                num_beams=8, 
                max_length=128
                )
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the token, and add the decoded texts with the references to the metric.
            decoded_summaries = [
                tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
                for s in summaries
            ]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        # Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        # Load tokenizer and model 
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # Load dataset
        samsum_dataset = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric(
            samsum_dataset['test'][0:10], 
            rouge_metric, 
            model, 
            tokenizer, 
            batch_size=2, 
            column_text='dialogue', 
            column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
        
        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_filename, index=False)

/Net/Groups/BGI/scratch/ppandey/miniconda3/envs/txtS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-04-28 23:42:16,142: INFO: config: PyTorch version 2.3.0+cu118 available.]


In [5]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-04-28 23:42:16,842: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-28 23:42:16,845: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-28 23:42:16,846: INFO: common: created directory at: artifacts]
[2024-04-28 23:42:16,847: INFO: common: created directory at: artifacts/model_evaluation]


/tmp/ipykernel_1360326/3261291991.py:84: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/Net/Groups/BGI/scratch/ppandey/miniconda3/envs/txtS/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 5/5 [00:07<00:00,  1.49s/it]

[2024-04-28 23:42:39,156: INFO: rouge_scorer: Using default tokenizer.]
